# Program Pre-processing vs No pre-processing

In [17]:
import pandas as pd
import autogluon
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

In [18]:
npp_data = pd.read_csv("/Users/ntix/Desktop/Programowanie/SUML/suml_15_grupa5 08-41-28-896/diabetes_dataset.csv")
pp_data = pd.read_csv("/Users/ntix/Desktop/Programowanie/SUML/suml_15_grupa5 08-41-28-896/diabetes_dataset_DONE")

In [19]:
from sklearn.feature_selection import mutual_info_classif

X = npp_data.drop("diagnosed_diabetes", axis=1)
y = npp_data["diagnosed_diabetes"]

npp_data.corr(numeric_only=True)["diagnosed_diabetes"].sort_values(ascending=False)

diagnosed_diabetes                    1.000000
hba1c                                 0.679397
glucose_postprandial                  0.629832
glucose_fasting                       0.510919
diabetes_risk_score                   0.277300
family_history_diabetes               0.197926
age                                   0.137713
bmi                                   0.097057
systolic_bp                           0.095481
waist_to_hip_ratio                    0.078918
ldl_cholesterol                       0.067475
cholesterol_total                     0.058173
insulin_level                         0.057715
triglycerides                         0.056230
diastolic_bp                          0.035619
cardiovascular_history                0.029793
hypertension_history                  0.027524
heart_rate                            0.022785
screen_time_hours_per_day             0.018127
alcohol_consumption_per_week          0.000760
sleep_hours_per_day                  -0.000399
diet_score   

In [20]:
# cols_to_drop = [
#     "family_history_diabetes",
#     "bmi",
#     "cholesterol_total",
#     "glucose_fasting",
#     "glucose_postprandial",
#     "diabetes_stage"
# ]

# npp_data = npp_data.drop(cols_to_drop, axis=1)

In [24]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X = npp_data.drop("diagnosed_diabetes", axis=1)
y = npp_data["diagnosed_diabetes"]

num_cols = X.select_dtypes(include=["number"]).columns
X_num = X[num_cols]

X_train, X_test, y_train, y_test = train_test_split(
    X_num, y, test_size=0.3, random_state=42, shuffle=True
)

pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca  = pca.transform(X_test)

ValueError: could not convert string to float: 'Male'

### 1. No pre-processing autogluon

In [21]:
train, test = train_test_split(
    npp_data,
    test_size=0.3,
    random_state=42,
    shuffle=True
)

train_data = TabularDataset(train)

predictor = TabularPredictor(label='diagnosed_diabetes', path='modelePPvsNPP/NPP').fit(train_data, presets="medium")

test_data = TabularDataset(test)

predictions = predictor.predict(test_data)
print(predictions)

leaderboard = predictor.leaderboard()
print(leaderboard)

print(predictor.evaluate(train_data))



print("----------hiper-----------")

model_name = predictor.model_best
model_info = predictor.info()
model_info['model_info'][model_name]['hyperparameters']

Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.5
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 22.6.0: Wed Jul  5 22:17:35 PDT 2023; root:xnu-8796.141.3~6/RELEASE_ARM64_T8112
CPU Count:          8
Memory Avail:       5.47 GB / 16.00 GB (34.2%)
Disk Space Avail:   786.66 GB / 926.35 GB (84.9%)
Presets specified: ['medium']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ...
AutoGluon will save models to "/Users/ntix/Desktop/Programowanie/SUML/AutoGluon/modelePPvsNPP/NPP"
Train Data Rows:    70000
Train Data Columns: 24
Label Column:       diagnosed_diabetes
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify

75721    0
80184    1
19864    0
76699    0
92991    0
        ..
42648    0
86306    0
45466    1
63724    0
34122    1
Name: diagnosed_diabetes, Length: 30000, dtype: int64
                  model  score_val eval_metric  pred_time_val   fit_time  \
0              LightGBM     0.9112    accuracy       0.001441   0.422177   
1   WeightedEnsemble_L2     0.9112    accuracy       0.001720   0.459363   
2               XGBoost     0.9108    accuracy       0.004410   0.382813   
3              CatBoost     0.9104    accuracy       0.001519   0.992072   
4      RandomForestGini     0.9104    accuracy       0.040633   3.738437   
5      RandomForestEntr     0.9104    accuracy       0.040752   3.748685   
6         LightGBMLarge     0.9100    accuracy       0.003547   2.315887   
7       NeuralNetFastAI     0.9068    accuracy       0.012079  26.660343   
8        NeuralNetTorch     0.9052    accuracy       0.010133  28.542751   
9            LightGBMXT     0.8996    accuracy       0.003243   0

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


{'use_orig_features': False,
 'valid_stacker': True,
 'max_base_models': 0,
 'max_base_models_per_type': 'auto',
 'save_bag_folds': True,
 'stratify': 'auto',
 'bin': 'auto',
 'n_bins': None}

### 2. Pre-processing autogluon

In [23]:
train, test = train_test_split(
    pp_data,
    test_size=0.3,
    random_state=42,
    shuffle=True
)

train_data = TabularDataset(train)

predictor = TabularPredictor(label='diagnosed_diabetes', path='modelePPvsNPP/NPP').fit(train_data, presets="medium")

test_data = TabularDataset(test)

predictions = predictor.predict(test_data)
print(predictions)

leaderboard = predictor.leaderboard()
print(leaderboard)

print(predictor.evaluate(train_data))



print("----------hiper-----------")

model_name = predictor.model_best
model_info = predictor.info()
model_info['model_info'][model_name]['hyperparameters']

Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.5
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 22.6.0: Wed Jul  5 22:17:35 PDT 2023; root:xnu-8796.141.3~6/RELEASE_ARM64_T8112
CPU Count:          8
Memory Avail:       4.83 GB / 16.00 GB (30.2%)
Disk Space Avail:   785.72 GB / 926.35 GB (84.8%)
Presets specified: ['medium']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ...
AutoGluon will save models to "/Users/ntix/Desktop/Programowanie/SUML/AutoGluon/modelePPvsNPP/NPP"
Train Data Rows:    58080
Train Data Columns: 16
Label Column:       diagnosed_diabetes
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1.0, 0.0]
	If 'binary' is not the correct problem_type, please manually spe

26892    1.0
79545    0.0
14554    0.0
51734    1.0
21371    1.0
        ... 
43079    0.0
26629    0.0
40625    0.0
29478    1.0
17817    1.0
Name: diagnosed_diabetes, Length: 24892, dtype: float64
                  model  score_val eval_metric  pred_time_val   fit_time  \
0              LightGBM     0.9080    accuracy       0.000629   0.397942   
1   WeightedEnsemble_L2     0.9080    accuracy       0.000931   0.435572   
2               XGBoost     0.9072    accuracy       0.001741   0.171319   
3         LightGBMLarge     0.9072    accuracy       0.002077   1.654974   
4              CatBoost     0.9068    accuracy       0.000329   0.287794   
5      RandomForestGini     0.9068    accuracy       0.037434   3.832963   
6      RandomForestEntr     0.9064    accuracy       0.039479   4.223098   
7        NeuralNetTorch     0.9044    accuracy       0.005798  29.869092   
8       NeuralNetFastAI     0.9036    accuracy       0.011221  21.266528   
9            LightGBMXT     0.9032    acc

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


{'use_orig_features': False,
 'valid_stacker': True,
 'max_base_models': 0,
 'max_base_models_per_type': 'auto',
 'save_bag_folds': True,
 'stratify': 'auto',
 'bin': 'auto',
 'n_bins': None}